In [1]:
import pandas as pd
import requests

C:\Users\amarl\AppData\Local\Temp\ipykernel_39540\1058106858.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [18]:
resp = requests.get('https://barttorvik.com/getgamestats.php?sIndex=7&year=2024&tvalue=All&cvalue=All&opcvalue=All&ovalue=All&minwin=All&mindate=11/06/2023&maxdate=11/07/2023&typev=All&venvalue=All&minadjo=0&minadjd=200&mintempo=0&minppp=0&minefg=0&mintov=200&minreb=0&minftr=0&minpppd=200&minefgd=200&mintovd=0&minrebd=200&minftrd=200&mings=0&mingscript=-100&maxx=100&coach=All&opcoach=All&adjoSelect=min&adjdSelect=max&tempoSelect=min&pppSelect=min&efgSelect=min&tovSelect=max&rebSelect=min&ftrSelect=min&pppdSelect=max&efgdSelect=max&tovdSelect=min&rebdSelect=max&ftrdSelect=max&gscriptSelect=min&sortToggle=1')

In [20]:
resp.json()

[['12/21/23',
  0,
  'Abilene Christian',
  'WAC',
  'Arkansas',
  'A',
  'L, 83-73',
  101.7,
  103.2,
  94.9,
  41.7,
  10.4,
  22.4,
  27.8,
  107.9,
  58.7,
  22.1,
  23.3,
  53.8,
  45.8,
  'SEC',
  1,
  2024,
  76.9,
  'Abilene ChristianArkansas12-21',
  'Brette Tanner',
  'Eric Musselman',
  -2.4428,
  0.694294,
  '["12/21/2023", 200, "Abilene Christian", "Arkansas", 28, 72, 4, 16, 13, 20, 11, 23, 34, 15, 10, 1, 8, 21, 73, 27, 52, 7, 15, 22, 28, 7, 38, 45, 9, 2, 6, 17, 17, 83, 76.9, "", "Arkansas", "Abilene Christian"]',
  1],
 ['12/21/23',
  0,
  'Arkansas',
  'SEC',
  'Abilene Christian',
  'H',
  'W, 83-73',
  104.4,
  104.5,
  107.9,
  58.7,
  22.1,
  23.3,
  53.8,
  94.9,
  41.7,
  10.4,
  22.4,
  27.8,
  49.7,
  'WAC',
  2,
  2024,
  76.9,
  'Abilene ChristianArkansas12-21',
  'Eric Musselman',
  'Brette Tanner',
  2.4428,
  0.26911,
  '["12/21/2023", 200, "Abilene Christian", "Arkansas", 28, 72, 4, 16, 13, 20, 11, 23, 34, 15, 10, 1, 8, 21, 73, 27, 52, 7, 15, 22, 28, 7, 38

In [21]:
print(resp.json()[0])

['12/21/23', 0, 'Abilene Christian', 'WAC', 'Arkansas', 'A', 'L, 83-73', 101.7, 103.2, 94.9, 41.7, 10.4, 22.4, 27.8, 107.9, 58.7, 22.1, 23.3, 53.8, 45.8, 'SEC', 1, 2024, 76.9, 'Abilene ChristianArkansas12-21', 'Brette Tanner', 'Eric Musselman', -2.4428, 0.694294, '["12/21/2023", 200, "Abilene Christian", "Arkansas", 28, 72, 4, 16, 13, 20, 11, 23, 34, 15, 10, 1, 8, 21, 73, 27, 52, 7, 15, 22, 28, 7, 38, 45, 9, 2, 6, 17, 17, 83, 76.9, "", "Arkansas", "Abilene Christian"]', 1]


In [22]:
header = ['Date', 'UNK1', 'Team', 'Conference', 'Opponent', 'Location', 'Result', 'Adj_Offense', 'Adj_Defense', 'Offensive_Efficiency',  'Effective_FG_Percent', 'Turnover_Percent', 'Offensive_Rebound_Percent', 'Free_Throw_Rate', 'Opponent_Offensive_Efficiency', 'Opponent_Effective_FG_Percent', 'Opponent_Turnover_Percent', 'Offensive_Rebound_Percent', 'Opponent_Free_Throw_Rate', 'UNK2', 'Opponent_Conference', 'UNK2', 'Season', 'UNK3', 'UNK4', 'UNK5', 'UNK6', 'UNK7', 'UNK8', 'UNK9', 'UNK10']

In [23]:
len(header) # same length, nice

31